In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import sys
sys.path.append('../YOLOv3')

In [ ]:
from models import *
from utils.logger import *
from utils.utils import *
from utils.datasets import *
from utils.parse_config import *
from val import evaluate

In [ ]:
from terminaltables import AsciiTable

In [ ]:
import os
import sys
import time
import datetime
import argparse

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim

In [ ]:
class Config:
    
    def __init__(self):
        self.epochs = 5
        self.batch_size = 7
        self.gradient_accumulations = 2
        self.model_def = '<base_path>/iWildCam2020/YOLOv3/config/yolov3.cfg'
        self.data_config = "<base_path>/YOLOv3/config/iWildCam.data"
        self.pretrained_weights = '<path_to_pretrained_models>'
        self.n_cpu = 2
        self.img_size = 512
        self.checkpoint_interval = 1
        self.evaluation_interval = 1
        self.compute_map = False
        self.multiscale_training = True
        self.lr = 1e-5
        
        self.log_dir = '<base_path>/iWildCam2020/YOLOv3/logs'
        self.output =  '<base_path>/iWildCam2020/YOLOv3/output'
        self.checkpoints =  '<base_path>/iWildCam2020/YOLOv3/checkpoints'
        
        self.device = 'cuda:0'
        self.multigpu = False
        
opt = Config()

In [ ]:
logger = Logger(opt.log_dir)

In [ ]:
os.makedirs(opt.output, exist_ok=True)
os.makedirs(opt.checkpoints, exist_ok=True)

In [ ]:
# Get data configuration
data_config = parse_data_config(opt.data_config)
train_path = data_config["train"]
valid_path = data_config["valid"]
class_names = load_classes(data_config["names"])

In [ ]:
# Initiate model
model = Darknet(opt.model_def)
model.apply(weights_init_normal)
# If specified we start from checkpoint
if opt.pretrained_weights:
    if opt.pretrained_weights.endswith(".pth"):
        model.load_state_dict(torch.load(opt.pretrained_weights))
    else:
        model.load_darknet_weights(opt.pretrained_weights)
if opt.multigpu:
    model = nn.DataParallel(model)
model = model.to(opt.device)

In [ ]:
# Get dataloader
dataset = ListDataset(train_path, augment=True, multiscale=opt.multiscale_training)
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=opt.batch_size,
    shuffle=True,
    num_workers=opt.n_cpu,
    pin_memory=True,
    collate_fn=dataset.collate_fn,
)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=opt.lr)

In [ ]:
metrics = [
    "grid_size",
    "loss",
    "x",
    "y",
    "w",
    "h",
    "conf",
    "cls",
    "cls_acc",
    "recall50",
    "recall75",
    "precision",
    "conf_obj",
    "conf_noobj",
    "classification_loss",
    "batch_acc"
]

In [ ]:
for epoch in range(4, opt.epochs):
    model.train()
    start_time = time.time()
    for batch_i, (_, imgs, targets, cls) in enumerate(dataloader):
        batches_done = len(dataloader) * epoch + batch_i

        imgs = Variable(imgs.to(opt.device))
        targets = Variable(targets.to(opt.device), requires_grad=False)
        cls = Variable(cls.to(opt.device), requires_grad=False)

        loss, outputs, class_scores = model(imgs, targets, cls)
        loss.backward()

        if batches_done % opt.gradient_accumulations:
            # Accumulates gradient before each step
            optimizer.step()
            optimizer.zero_grad()

        # ----------------
        #   Log progress
        # ----------------

        log_str = "\n---- [Epoch %d/%d, Batch %d/%d] ----\n" % (epoch, opt.epochs, batch_i, len(dataloader))

        metric_table = [["Metrics", *[f"YOLO Layer {i}" for i in range(len(model.yolo_layers))]]]

        # Log metrics at each YOLO layer
        for i, metric in enumerate(metrics):
            formats = {m: "%.6f" for m in metrics}
            formats["grid_size"] = "%2d"
            formats["cls_acc"] = "%.2f%%"
            row_metrics = [formats[metric] % yolo.metrics.get(metric, 0) for yolo in model.yolo_layers]
            metric_table += [[metric, *row_metrics]]

            # Tensorboard logging
            tensorboard_log = []
            for j, yolo in enumerate(model.yolo_layers):
                for name, metric in yolo.metrics.items():
                    if name != "grid_size":
                        tensorboard_log += [(f"{name}_{j+1}", metric)]
            tensorboard_log += [("loss", loss.item())]
            logger.list_of_scalars_summary(tensorboard_log, batches_done)

        log_str += AsciiTable(metric_table).table
        log_str += f"\nTotal loss {loss.item()}"

        # Determine approximate time left for epoch
        epoch_batches_left = len(dataloader) - (batch_i + 1)
        time_left = datetime.timedelta(seconds=epoch_batches_left * (time.time() - start_time) / (batch_i + 1))
        log_str += f"\n---- ETA {time_left}"
    
        if batches_done % 50 == 0:
            print(log_str)

        model.seen += imgs.size(0)
    if epoch % opt.evaluation_interval == 0:
        print("\n---- Evaluating Model ----")
        # Evaluate the model on the validation set
        precision, recall, AP, f1, ap_class, acc = evaluate(
            model,
            path=valid_path,
            iou_thres=0.5,
            conf_thres=0.5,
            nms_thres=0.5,
            img_size=opt.img_size,
            batch_size=8,
        )
        evaluation_metrics = [
            ("val_precision", precision.mean()),
            ("val_recall", recall.mean()),
            ("val_mAP", AP.mean()),
            ("val_f1", f1.mean()),
            ("val_acc", acc)
        ]
        logger.list_of_scalars_summary(evaluation_metrics, epoch)

        # Print class APs and mAP
        ap_table = [["Index", "Class name", "AP"]]
        for i, c in enumerate(ap_class):
            ap_table += [[c, class_names[c], "%.5f" % AP[i]]]
        print(AsciiTable(ap_table).table)
        print(f"---- mAP {AP.mean()}")
        print(f"---- Val Accuracy {acc}")

    if epoch % opt.checkpoint_interval == 0:
        torch.save(model.state_dict(), f"../YOLOv3/checkpoints/yolov3_ckpt_%d.pth" % epoch)
    
    logger.write_log(epoch)

In [ ]:
# torch.save(model.state_dict(), f"../YOLOv3/checkpoints/yolov3_ckpt_%d.pth" % epoch)

In [ ]:
# logger.write_log(epoch)